# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [4]:
pd_data.shape[0]

14

## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [5]:
def get_gini(df, label):
    
    ss = df[label].value_counts()
    s_label = ss.index
    s_cc = ss.values # 각각 클래쓰별 개수
    s_sum = s_cc.sum() #전체 개수
    
    gini = 1
    for value in s_cc : # 지니함수 공식 그대로 적용하기 
        gini -= (value/s_sum)**2
        
        
    return gini

In [6]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [8]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    result = []
    
    for i in range(1, len(uniques)):
        
        result += [list(x) for x in combinations(uniques, i)]
    
    return result

In [9]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for sp in binary_split:
        
        tr = df[df[attribute].isin(sp)]
        nr = df[-df[attribute].isin(sp)]
        gini = (len(tr) * get_gini(tr, label) + len(nr) *get_gini(nr, label)) /len(df)
        result['_'.join(sp)] = gini
    
    return result

In [11]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.45833333333333337,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.45833333333333337,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [12]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [13]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [14]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print(feature,min(get_attribute_gini_index(pd_data, feature, label).values()))

age 0.35714285714285715
income 0.4428571428571429
student 0.3673469387755103
credit_rating 0.42857142857142855


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [15]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.39365079365079364,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [18]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    
    entro = 0 #초기화
    unique = list(df[feature].unique())
    
    
    for i in unique:
        
        final = sum(df[feature] == i) / len(df[feature]) # 잔체 개수에서 특정 feature를 나누어준다. 
        entro -= final * np.log2(final) # 공식 그대로 적용하기 
        
    return entro
                    


In [19]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [20]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    info = getEntropy(df,feature)
    final_feature = list(df.loc[:,df.columns!= feature])
    gain = []
    result = {}

    for column in final_feature:
        infoi = 0
        new_feature = df[column]
        for i in new_feature.unique():
            all_final = df.loc[new_feature == i]
            
            info += (len(all_final)/len(new_feature)) * getEntropy(all_final, feature)
            
        gain.append(info - infoi)
        
    result = dict(zip(final_feature, gain))

In [22]:
getGainA(pd_data, "class_buys_computer")